# Date Night Movie

#### Grading:


- Code: 90 pts
- Markdown Documentation: 10 pts


In this assignment we are going to use pandas to figure out - What's the best **date-night movie**?

This assignment is going to use
- Joining
- Groupby
- Sorting


In [25]:
import os
import pandas as pd

##### Read in the movie data: `pd.read_table`

In [26]:
def get_movie_data():
    
    unames = ['user_id','gender','age','occupation','zip']
    users = pd.read_table(os.path.join('../data','users.dat'), 
                          sep='::', header=None, names=unames)
    
    rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
    ratings = pd.read_table(os.path.join('../data', 'ratings.dat'), 
                            sep='::', header=None, names=rnames)
    
    mnames = ['movie_id', 'title','genres']
    movies = pd.read_table(os.path.join('../data', 'movies.dat'), 
                           sep='::', header=None, names=mnames)

    return users, ratings, movies

In [27]:
users, ratings, movies = get_movie_data()

/usr/local/lib64/python3.8/site-packages/pandas/io/parsers.py:755: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())


In [28]:
print(users.head())

   user_id gender  age  occupation    zip
0        1      F    1          10  48067
1        2      M   56          16  70072
2        3      M   25          15  55117
3        4      M   45           7  02460
4        5      M   25          20  55455


In [29]:
print(ratings.head())

   user_id  movie_id  rating  timestamp
0        1      1193       5  978300760
1        1       661       3  978302109
2        1       914       3  978301968
3        1      3408       4  978300275
4        1      2355       5  978824291


In [30]:
print(movies.head())

   movie_id                               title                        genres
0         1                    Toy Story (1995)   Animation|Children's|Comedy
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy
2         3             Grumpier Old Men (1995)                Comedy|Romance
3         4            Waiting to Exhale (1995)                  Comedy|Drama
4         5  Father of the Bride Part II (1995)                        Comedy


##### Clean up the `movies`

- Get the `year`
- Shorten the `title`


In [31]:
tmp = movies.title.str.extract('(.*) \(([0-9]+)\)')
tmp.apply(lambda x:x[0] if len(x) > 0 else None)
tmp.apply(lambda x: x[0][:40] if len(x) > 0 else None)

0    Toy Story
1         1995
dtype: object

In [34]:
movies['year'] = tmp[1]
movies['short_title'] = tmp[0]

In [35]:
print(movies.head())

   movie_id                               title                        genres  \
0         1                    Toy Story (1995)   Animation|Children's|Comedy   
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy   
2         3             Grumpier Old Men (1995)                Comedy|Romance   
3         4            Waiting to Exhale (1995)                  Comedy|Drama   
4         5  Father of the Bride Part II (1995)                        Comedy   

   year                  short_title  
0  1995                    Toy Story  
1  1995                      Jumanji  
2  1995             Grumpier Old Men  
3  1995            Waiting to Exhale  
4  1995  Father of the Bride Part II  


##### Join the tables with `pd.merge` (10 pts)

In [36]:
import numpy as np
df_user_rating = pd.merge(users, ratings, how = "inner", on = "user_id")
df_user_rating_movie = pd.merge(df_user_rating, movies, how = "inner",\
                                 on = "movie_id")
df_user_rating_movie

,user_id,gender,age,occupation,zip,movie_id,rating,timestamp,title,genres,year,short_title
0,1,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,1975,One Flew Over the Cuckoo's Nest
1,2,M,56,16,70072,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama,1975,One Flew Over the Cuckoo's Nest
2,12,M,25,12,32793,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama,1975,One Flew Over the Cuckoo's Nest
3,15,M,25,7,22903,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama,1975,One Flew Over the Cuckoo's Nest
4,17,M,50,1,95350,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama,1975,One Flew Over the Cuckoo's Nest
...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,5949,M,18,17,47901,2198,5,958846401,Modulations (1998),Documentary,1998,Modulations
1000205,5675,M,35,14,30030,2703,3,976029116,Broken Vessels (1998),Drama,1998,Broken Vessels
1000206,5780,M,18,17,92886,2845,1,958153068,White Boys (1999),Drama,1999,White Boys
1000207,5851,F,18,20,55410,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western,1973,One Little Indian


##### What's the highest rated movie? (10 pts))

In [178]:
# Can we groupby prior to getting the ratings to make things easier.
# Average rating
# Average Rating and max number of ratings (how many ratings did it get)
# Can we do the intersection of median and mean? Is this useful?
# Should get only one movie once we factor in ratings.

df_mean_rating = pd.DataFrame(columns = ["movie_id", "mean_rating", "rating_count",\
                                         "count_weight", "weighted"])
df_mean_rating.set_index("movie_id", inplace = True)

for x in df_user_rating_movie.groupby(by = "movie_id"):
    df_mean_rating.loc[x[0]] = [x[1].rating.mean(), x[1].rating.count(),\
                                np.log2(x[1].rating.count()), x[1].rating.mean() * np.log2(x[1].rating.count())]
df_mean_rating

,mean_rating,rating_count,count_weight,weighted
movie_id,,,,
1,4.146846,2077.0,11.020286,45.699431
2,3.201141,701.0,9.453271,30.261254
3,3.016736,478.0,8.900867,26.851569
4,2.729412,170.0,7.409391,20.223279
5,3.006757,296.0,8.209453,24.683829
...,...,...,...,...
3948,3.635731,862.0,9.751544,35.453990
3949,4.115132,304.0,8.247928,33.941307
3950,3.666667,54.0,5.754888,21.101254


In [181]:
max_rating = df_mean_rating.mean_rating.max()
df_top_ratings = df_mean_rating[df_mean_rating.mean_rating == max_rating]
df_top_ratings

,mean_rating,rating_count,count_weight,weighted
movie_id,,,,
787,5.0,3.0,1.584963,7.924813
989,5.0,1.0,0.000000,0.000000
1830,5.0,1.0,0.000000,0.000000
3172,5.0,1.0,0.000000,0.000000
3233,5.0,2.0,1.000000,5.000000
3280,5.0,1.0,0.000000,0.000000
3382,5.0,1.0,0.000000,0.000000
3607,5.0,1.0,0.000000,0.000000
3656,5.0,1.0,0.000000,0.000000


In [207]:
max_ratings = df_mean_rating[df_mean_rating.mean_rating >= 3.74]
max_ratings

,mean_rating,rating_count,count_weight,weighted
movie_id,,,,
1,4.146846,2077.0,11.020286,45.699431
6,3.878723,940.0,9.876517,38.308277
11,3.793804,1033.0,10.012625,37.985940
16,3.793255,682.0,9.413628,35.708292
17,4.027545,835.0,9.705632,39.089870
...,...,...,...,...
3929,3.993197,147.0,7.199672,28.749712
3932,3.750000,232.0,7.857981,29.467429
3949,4.115132,304.0,8.247928,33.941307


In [208]:
df_mean_rating.mean_rating.quantile(0.75)
#max_ratings.rating_count.var()

3.740740740740741

In [209]:
df_mean_rating.rating_count.quantile(0.75)
df_75_count = df_mean_rating[df_mean_rating.rating_count >= 350]

In [210]:
top_ratings = df_75_count.merge(max_ratings, how = "inner")
top_ratings

,mean_rating,rating_count,count_weight,weighted
0,4.146846,2077.0,11.020286,45.699431
1,3.878723,940.0,9.876517,38.308277
2,3.793804,1033.0,10.012625,37.985940
3,3.793255,682.0,9.413628,35.708292
4,4.027545,835.0,9.705632,39.089870
...,...,...,...,...
399,3.909333,375.0,8.550747,33.427719
400,4.226358,994.0,9.957102,42.082279
401,4.073059,657.0,9.359750,38.122816
402,3.924485,437.0,8.771489,34.423580


In [218]:
top_rated = df_mean_rating[df_mean_rating.weighted >= df_mean_rating.weighted.max() - 1]

In [219]:
movies[movies.movie_id == 260]

,movie_id,title,genres,year,short_title
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1977,Star Wars: Episode IV - A New Hope


In [220]:
movies[movies.movie_id.isin(list(top_rated.index.values))]

,movie_id,title,genres,year,short_title
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1977,Star Wars: Episode IV - A New Hope
315,318,"Shawshank Redemption, The (1994)",Drama,1994,"Shawshank Redemption, The"
1180,1198,Raiders of the Lost Ark (1981),Action|Adventure,1981,Raiders of the Lost Ark
2789,2858,American Beauty (1999),Comedy|Drama,1999,American Beauty


In [221]:
ratings.head()
ratings.movie_id[777]

912

###### What is a good rated movie for date night? (30 pts)

- Hint - highly rated movie by 
    - both partners (might be the same gender or not),
    - based on genre preferences,
    - age group can also be combined

In [215]:
users

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060
